In [1]:
from IPython.display import display, HTML
import numpy as np
import re
div_to_color = {'0': '#FFFFFF',
                '1': '#e0e0e0',
                '2': '#DFFF00',
                '3': '#9FE2BF',
                '4': '#CCCCFF',
                '5': '#FFE4E1',
                '6': '#FF00FF',
                '7': '#e3abfb',    
                '8': '#FFBF00',
                '9': '#BBBAFF',
                '10': '#FFC2E2',
                '11': '#F4FDB1',
                '12': '#7C8BFF',
                '13': '#95FFF7',
                '14': '#FF5733',
                '15': '#33FF4C',
                '16': '#7733FF',
                '17': '#B2FF33',
                '18': '#33FF83', 
                '19': '#FF3371', 
                '20': '#33F3FF',
                '21': '#8AFF33',
                '22': '#33FFC1',
                '23': '#AF33FF',
                '24': '#FFD433',
                '25': '#33ACFF',
                '26': '#BDAACF',
                '27': '#ACE6DA',
                '28': '#CCCCFF',
                '29': '#FFE4E1',
                '30': '#FF00FF',
                '31': '#e3abfb',    
                '32': '#FFBF00',
                '33': '#BBBAFF',
}

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def color_html(label):
    return f'<span style="background-color:{div_to_color[label]} ">'

def display_HTML(text, div, offset=False, one_color=False, red_chars=False):
    displayed_text = ''
    user_displayed_text = ''
    previous = 'eq'
    mark_end = '</span>'
    previous_label = 0
    for token, label in zip(text.split(' '), div.split(' ')):
        #print(label)
        if one_color:
            if int(label) > 0:
                label = '1'
        if offset:
            if label != '0':
                label = str(int(label)  + 1)
        mark = color_html(label)
        if label == '0':
            displayed_text += f' {token}'
            if previous == 'div':
                user_displayed_text += f'</h-l-{str(int(previous_label)-1)}> {token}'
            else:
                user_displayed_text += f' {token}'
            previous = 'eq'
        else:
            if previous == 'div' and int(label)==previous_label:
                displayed_text += f'{mark} {token}{mark_end}'
                user_displayed_text += f' {token}'
            else:
                if int(label)!=previous_label and int(label) > 2 and previous_label!=0:
                    user_displayed_text += f' </h-l-{str(int(label)-2)}>'
                displayed_text += f' {mark}{token}{mark_end}'
                user_displayed_text += f' <h-l-{str(int(label)-1)}>{token}'
            previous = 'div'
        previous_label = int(label)

    if red_chars:
        displayed_text = f'{bcolors.WARNING} {displayed_text} {bcolors.ENDC}'
    
    if previous == 'div':
        user_displayed_text += f'</h-l-{str(int(previous_label)-1)}>'
        
        
    return display(HTML(displayed_text)), user_displayed_text

def add_lists(list_a, list_b):
    #print(list_b)
    #print(list_a)
    #print('as')
    list_add = []
    for (a, b) in zip(list_a, list_b):
        list_add.append(a+b)
    return list_add

In [2]:
processes, processes_set = [], set()
with open('annotations/limsi', 'r') as file_:
    lines = file_.readlines()
    for line in lines:
        line = line.rstrip().split('\t')[-1]
        if '0' in line:
            processes.append(['literal'])
        else:
            processes.append(line.split(' '))
        for a  in line.split(' '):
            processes_set.add(a)

In [3]:
import random
random.seed(41)

o, g = [],[]
count = 0
dir_='en_pt_ced'
#dir_='en_de_ced'

meta_id, meta_lbl, out_src, out_tgt = f'for_user_study/{dir_}/meta_id', f'for_user_study/{dir_}/meta_lbl', f'for_user_study/{dir_}/src', f'for_user_study/{dir_}/tgt'
original_scores, final_scores =[], []

bad, explained =0, 0
no_highlights_ok = 0
halluci, no_halluci= 0, 0

lbls = open(f'outputs/test.2022.pt-en.label', 'r').readlines()
#lbls = open(f'outputs/test.2022.en-de.label', 'r').readlines()
diffs = []
original_count = 0
original_scores_ced = []
gpt_count = 0
bad_num, num_diff, total = 0, 0, 0
for file_ in ['en_pt_ced_test_missing_paired_phrases_reward_pos']:

    with open(f'outputs/{file_}', 'r') as out_:
        c = 0
        lines = out_.readlines()
        pop = len(lines)
        sample = random.sample(range(0, len(lines)), len(lines))
        #sample = random.sample(range(1, len(lines)), 100)
        #print(sample)
        for id_, line in enumerate(lines):
            
            #if id_ not in sample:
            #    continue
            lbl =lbls[id_].rstrip()
            #lbl = 'original'
            scores, score_color = [], []
                
            line = line.split('\t')
            
            try:
                raw_a = np.array([float(x) for x in line[5].split(' ')])
                raw_b = np.array([float(x) for x in line[6].split(' ')])
            except IndexError:
                continue
                #print('Problem')
            
            if len(line[0].split(' ')) < 15:
                continue
            total+=1
        
            original_score = float(line[4])
            revisions = int(line[5])
            original_scores.append(original_score)        
            previous_score = original_score
            expl_a = [0] * len(line[0].split(' '))
            expl_b = [0] * len(line[1].split(' '))
                    
            scores.append(str(original_score))
            score_color = '1'
                    
           
            for i in range(revisions):                        
                score = line[5 + 3*i + 1]               
                #if float(score) - float(previous_score) < 5:
                #if float(score) - float(previous_score) < 2 or original_score > 10:
                #    break
                if i == 0:
                    bar = 1
                else:
                    bar = 1
                if float(previous_score) > 10:
                    break
                if float(score) - float(previous_score) < bar or original_score > 10:
                    break
                else:
                    #lbl='chatGPT_paraphrase'
                    if i==0:
                        count+=1
                expl_a = add_lists(expl_a, [((i +1)*int(a)) for a in line[5 + 3*i + 2].split(' ')])
                expl_b = add_lists(expl_b, [((i +1)*int(a)) for a in line[5 + 3*i + 3].split(' ')])
    
                previous_score = score
                scores.append(str(score))
                score_color += f' {str(i+2)}'
                
            final_scores.append(float(scores[-1]))
            diff = float(scores[-1]) - float(original_score) 

                        
            s_ = 0
            if diff <=0: 
                if float(scores[0]) < 0:
                    expl_a =  [1]*len(line[0])
                    expl_b =  [1]*len(line[1])
                #continue
                
            en_num = [int(i) for i in line[0].split() if i.isdigit()]
            nen_num = [int(i) for i in line[1].split() if i.isdigit()]

            if lbl == 'BAD':
                continue
            if original_score > 0:
                continue
            else:
                bad_num +=1
                src_num = re.findall(r'\d+', line[0])
                tgt_num = re.findall(r'\d+', line[1])
                if set(src_num) != set(tgt_num):
                    continue
            num_diff +=1
            original_scores_ced.append(float(original_score))
            print(f'____________________________________{id_}___________________________________________________')
            
            if lbl == 'BAD' or lbl=='1': 
                if ([a.isupper() for a in line[0]].count(True))/len(line[0]) > 0.5 or '� � ' in line[0]:
                    halluci+=1
                #continue
                if set(en_num) != set(nen_num) and sum(expl_a)==0 and sum(expl_b)==0:
                    print('Numeric difference!!')
                    #print(en_num)
                    #print(nen_num)
                    explained+=1
                    #continue
                #print(nen_num)
                bad+=1
                if sum(expl_a) !=0:
                    explained+=1
                print(bcolors.FAIL + bcolors.BOLD + "😤 😠 😡 🤬  DETACHED HALLUCINATION" + bcolors.ENDC)
                print(diff)
            else:
                if ([a.isupper() for a in line[0]].count(True))/len(line[0]) <= 0.5 or '� � ' not in line[0]:
                    no_halluci+=1

                if sum(expl_a) + sum(expl_b) ==0:
                    no_highlights_ok+=1
                #continue
                print(bcolors.OKBLUE + bcolors.BOLD + "🤓 OK" + bcolors.ENDC)
            
            #print(c)

            
            display_HTML(' '.join(scores), ' '.join(score_color.split(' ')))            
            diffs.append(diff)
            
            _, source_text = display_HTML(line[0], ' '.join([str(x) for x in expl_a]), offset=True)
            _, target_text = display_HTML(line[1], ' '.join([str(x) for x in expl_b]), offset=True) 
            
            
            if lbl == 'original':
                original_count +=1
            else:
                gpt_count += 1
            if original_count > 30 and lbl=='original':
                continue
                
            c+=1
            
            o.append(lbl)
            continue
            meta_id_file = open(f'{meta_id}_{c}.txt', 'w')
            meta_lbl_file = open(f'{meta_lbl}_{c}.txt', 'w')
            out_src_file = open(f'{out_src}_{c}.txt', 'w')
            out_tgt_file = open(f'{out_tgt}_{c}.txt', 'w')

            meta_id_file.write(f'{id_}')
            meta_lbl_file.write(f'{lbl}')
            out_src_file.write(f'{source_text}')
            out_tgt_file.write(f'{target_text}')

            meta_id_file.close()
            meta_lbl_file.close()
            out_src_file.close()
            out_tgt_file.close()
            

____________________________________1___________________________________________________
🤓 OK


____________________________________26___________________________________________________
🤓 OK


____________________________________29___________________________________________________
🤓 OK


____________________________________30___________________________________________________
🤓 OK


____________________________________31___________________________________________________
🤓 OK


...

____________________________________465___________________________________________________
🤓 OK


____________________________________467___________________________________________________
🤓 OK


____________________________________473___________________________________________________
🤓 OK


____________________________________477___________________________________________________
🤓 OK


____________________________________485___________________________________________________
🤓 OK


In [16]:
import random
from collections import defaultdict
random.seed(41)

o, g = [],[]
count = 0
dir_='en_pt_ced'
#dir_='en_de_ced'

batches_num = 8

meta_id, meta_lbl, out_src, out_tgt = f'for_user_study/{dir_}/meta_id', f'for_user_study/{dir_}/meta_lbl', f'for_user_study/{dir_}/src', f'for_user_study/{dir_}/tgt'
original_scores, final_scores =[], []

bad, explained =0, 0
no_highlights_ok = 0
halluci, no_halluci= 0, 0
original, halluc = defaultdict(list), defaultdict(list)
original_neg= defaultdict(list)
lbls = open(f'outputs/test.2022.pt-en.label', 'r').readlines()
#lbls = open(f'outputs/test.2022.en-de.label', 'r').readlines()
diffs = []
original_count = 0
original_scores_ced = []
gpt_count = 0
length =[]
bad_num, num_diff, total = 0, 0, 0
for file_ in ['en_pt_ced_test_missing_paired_phrases_reward_pos']:

    with open(f'outputs/{file_}', 'r') as out_:
        c = 0
        lines = out_.readlines()
        pop = len(lines)
        sample = random.sample(range(0, len(lines)), len(lines))
        #sample = random.sample(range(1, len(lines)), 100)
        #print(sample)
        for id_, line in enumerate(lines):
            
            #if id_ not in sample:
            #    continue
            lbl =lbls[id_].rstrip()
            #lbl = 'original'
            scores, score_color = [], []
                
            line = line.split('\t')
            try:
                raw_a = np.array([float(x) for x in line[5].split(' ')])
                raw_b = np.array([float(x) for x in line[6].split(' ')])
            except IndexError:
                continue
                #print('Problem')
            
            if len(line[0].split(' ')) < 15 or len(line[0].split(' ')) > 40:
                continue
            total+=1
        
            original_score = float(line[4])
            revisions = int(line[5])
            original_scores.append(original_score)        
            previous_score = original_score
            expl_a = [0] * len(line[0].split(' '))
            expl_b = [0] * len(line[1].split(' '))
            if lbl =='BAD' and original_score>10:
                continue
                    
            scores.append(str(original_score))
            score_color = '1'
                    
           
            for i in range(revisions):                        
                score = line[5 + 3*i + 1]               
                #if float(score) - float(previous_score) < 5:
                #if float(score) - float(previous_score) < 2 or original_score > 10:
                #    break
                if i == 0:
                    bar = 1
                else:
                    bar = 1
                if float(previous_score) > 10:
                    break
                if float(score) - float(previous_score) < bar or original_score > 10:
                    break
                else:
                    #lbl='chatGPT_paraphrase'
                    if i==0:
                        a=1
                        #count+=1
                expl_a = add_lists(expl_a, [((i +1)*int(a)) for a in line[5 + 3*i + 2].split(' ')])
                expl_b = add_lists(expl_b, [((i +1)*int(a)) for a in line[5 + 3*i + 3].split(' ')])
    
                previous_score = score
                scores.append(str(score))
                score_color += f' {str(i+2)}'
                
            final_scores.append(float(scores[-1]))
            diff = float(scores[-1]) - float(original_score) 

                        
            s_ = 0
            if diff <=0: 
                if float(scores[0]) < 0:
                    expl_a =  [1]*len(line[0])
                    expl_b =  [1]*len(line[1])
                #continue
                
            en_num = [int(i) for i in line[0].split() if i.isdigit()]
            nen_num = [int(i) for i in line[1].split() if i.isdigit()]

            if lbl != 'BAD':
                a=1
                #continue
            else:
                bad_num +=1
                src_num = re.findall(r'\d+', line[0])
                tgt_num = re.findall(r'\d+', line[1])
                if set(src_num) != set(tgt_num):
                    a=1
                    continue
            num_diff +=1
            original_scores_ced.append(float(original_score))
            print(f'____________________________________{id_}___________________________________________________')
            
            if lbl == 'BAD' or lbl=='1': 
                if ([a.isupper() for a in line[0]].count(True))/len(line[0]) > 0.5 or '� � ' in line[0]:
                    halluci+=1
                #continue
                if set(en_num) != set(nen_num) and sum(expl_a)==0 and sum(expl_b)==0:
                    print('Numeric difference!!')
                    #print(en_num)
                    #print(nen_num)
                    explained+=1
                    #continue
                #print(nen_num)
                bad+=1
                if sum(expl_a) !=0:
                    explained+=1
                print(bcolors.FAIL + bcolors.BOLD + "😤 😠 😡 🤬  DETACHED HALLUCINATION" + bcolors.ENDC)
                print(diff)
            else:
                if ([a.isupper() for a in line[0]].count(True))/len(line[0]) <= 0.5 or '� � ' not in line[0]:
                    no_halluci+=1

                if sum(expl_a) + sum(expl_b) ==0:
                    no_highlights_ok+=1
                #continue
                print(bcolors.OKBLUE + bcolors.BOLD + "🤓 OK" + bcolors.ENDC)
            
            #print(c)

            
            display_HTML(' '.join(scores), ' '.join(score_color.split(' ')))            
            diffs.append(diff)
            count+=1
            
            _, source_text = display_HTML(line[0], ' '.join([str(x) for x in expl_a]), offset=True)
            _, target_text = display_HTML(line[1], ' '.join([str(x) for x in expl_b]), offset=True) 
            
            
            if lbl == 'original':
                original_count +=1
            else:
                gpt_count += 1
            if original_count > 30 and lbl=='original':
                continue
                
            c+=1
            
            o.append(lbl)
            
            if lbl != 'BAD':
                if original_score > 5:
                    original[id_].append([source_text, target_text])
                else:
                    original_neg[id_].append([source_text, target_text])
            else:
                if float(score) < 0 or original_score >0:
                    continue
                halluc[id_].append([source_text, target_text])
            length.append(len(line[0].split(' ')))


            continue
            meta_id_file = open(f'{meta_id}_{c}.txt', 'w')
            meta_lbl_file = open(f'{meta_lbl}_{c}.txt', 'w')
            out_src_file = open(f'{out_src}_{c}.txt', 'w')
            out_tgt_file = open(f'{out_tgt}_{c}.txt', 'w')

            meta_id_file.write(f'{id_}')
            meta_lbl_file.write(f'{lbl}')
            out_src_file.write(f'{source_text}')
            out_tgt_file.write(f'{target_text}')

            meta_id_file.close()
            meta_lbl_file.close()
            out_src_file.close()
            out_tgt_file.close()
        c

____________________________________0___________________________________________________
🤓 OK


____________________________________2___________________________________________________
🤓 OK


____________________________________3___________________________________________________
😤 😠 😡 🤬  DETACHED HALLUCINATION
7.477621


____________________________________5___________________________________________________
🤓 OK


____________________________________7___________________________________________________
😤 😠 😡 🤬  DETACHED HALLUCINATION
2.0173119999999995


...

____________________________________493___________________________________________________
🤓 OK


____________________________________494___________________________________________________
🤓 OK


____________________________________496___________________________________________________
🤓 OK


____________________________________497___________________________________________________
🤓 OK


____________________________________498___________________________________________________
🤓 OK


In [17]:
halluc_dif = defaultdict(list)
for file_ in ['en_pt_ced_test_with_labels_missing_paired_phrases_reward_pos']:

    with open(f'outputs/{file_}', 'r') as out_:
        c = 0
        lines = out_.readlines()
        pop = len(lines)
        sample = random.sample(range(0, len(lines)), len(lines))
    
        for id_, line in enumerate(lines):
            
            if id_==9:
                continue
            
      
            scores, score_color = [], []
                
            line = line.split('\t')
            try:
                raw_a = np.array([float(x) for x in line[5].split(' ')])
                raw_b = np.array([float(x) for x in line[6].split(' ')])
            except IndexError:
                continue
                #print('Problem')

            total+=1
        
            original_score = float(line[4])
            revisions = int(line[5])
            previous_score = original_score
            expl_a = [0] * len(line[0].split(' '))
            expl_b = [0] * len(line[1].split(' '))

                    
            score_color = '1'
                    
           
            for i in range(revisions):                        
                score = line[5 + 3*i + 1]               
                #if float(score) - float(previous_score) < 5:
                #if float(score) - float(previous_score) < 2 or original_score > 10:
                #    break
                if i == 0:
                    bar = 1
                else:
                    bar = 1
                if float(previous_score) > 10:
                    break
                if float(score) - float(previous_score) < bar or original_score > 10:
                    break
                else:
                    #lbl='chatGPT_paraphrase'
                    if i==0:
                        a=1
                        #count+=1
                expl_a = add_lists(expl_a, [((i +1)*int(a)) for a in line[5 + 3*i + 2].split(' ')])
                expl_b = add_lists(expl_b, [((i +1)*int(a)) for a in line[5 + 3*i + 3].split(' ')])
    
                previous_score = score
                scores.append(str(score))
                score_color += f' {str(i+2)}'
                
            final_scores.append(float(scores[-1]))
            diff = float(scores[-1]) - float(original_score) 

                        
            s_ = 0
            if diff <=0: 
                if float(scores[0]) < 0:
                    expl_a =  [1]*len(line[0])
                    expl_b =  [1]*len(line[1])
                #continue
                
            en_num = [int(i) for i in line[0].split() if i.isdigit()]
            nen_num = [int(i) for i in line[1].split() if i.isdigit()]

            print(f'____________________________________{id_}___________________________________________________')
            
            if lbl == 'BAD' or lbl=='1': 
                if ([a.isupper() for a in line[0]].count(True))/len(line[0]) > 0.5 or '� � ' in line[0]:
                    halluci+=1
                #continue
                if set(en_num) != set(nen_num) and sum(expl_a)==0 and sum(expl_b)==0:
                    print('Numeric difference!!')
                    #print(en_num)
                    #print(nen_num)
                    explained+=1
                    #continue
                #print(nen_num)
                bad+=1
                if sum(expl_a) !=0:
                    explained+=1
                print(bcolors.FAIL + bcolors.BOLD + "😤 😠 😡 🤬  DETACHED HALLUCINATION" + bcolors.ENDC)
                print(diff)
            else:
                if ([a.isupper() for a in line[0]].count(True))/len(line[0]) <= 0.5 or '� � ' not in line[0]:
                    no_halluci+=1

                if sum(expl_a) + sum(expl_b) ==0:
                    no_highlights_ok+=1
                #continue
                print(bcolors.OKBLUE + bcolors.BOLD + "🤓 OK" + bcolors.ENDC)
            
            #print(c)

            
            display_HTML(' '.join(scores), ' '.join(score_color.split(' ')))            
            diffs.append(diff)
            count+=1
            
            _, source_text = display_HTML(line[0], ' '.join([str(x) for x in expl_a]), offset=True)
            _, target_text = display_HTML(line[1], ' '.join([str(x) for x in expl_b]), offset=True) 
            
                  
            halluc_dif[id_].append([source_text, target_text])

____________________________________0___________________________________________________
🤓 OK


____________________________________1___________________________________________________
🤓 OK


____________________________________2___________________________________________________
🤓 OK


____________________________________3___________________________________________________
🤓 OK


____________________________________4___________________________________________________
🤓 OK


...

____________________________________6___________________________________________________
🤓 OK


____________________________________7___________________________________________________
🤓 OK


____________________________________8___________________________________________________
🤓 OK


____________________________________10___________________________________________________
🤓 OK


____________________________________11___________________________________________________
🤓 OK


In [20]:
len(halluc)

17

In [19]:
import os
shift_o, shift_h, shift_n, shift_d = 0, 0, 0,0
batches_num = 2
for i in range(batches_num):
    c = 0
    print('\n------')
    dir_=f'gpt_lit_en_pt_ced_{i}'
    os.makedirs(f'for_user_study/{dir_}', exist_ok=True)
    meta_id, meta_lbl, out_src, out_tgt = f'for_user_study/{dir_}/meta_id', f'for_user_study/{dir_}/meta_lbl', f'for_user_study/{dir_}/src', f'for_user_study/{dir_}/tgt'

    
    original_items = original.items()
    orig_chun = list(original_items)[shift_o:15 + shift_o]
    
    #original_neg_items = original_neg.items()
    #orig_neg_chun = list(original_neg_items)[shift_o:5 + shift_o]
    
    halluc_items = halluc.items()
    hal_chun = list(halluc_items)[shift_h : 5 + shift_h]

    
    halluc_diff_items = halluc_dif.items()
    hal_diff_chun = list(halluc_diff_items)[shift_d : 5 + shift_d]
    #print(hal_chun)
    #print('\n')
    
    shift_o += 15
    #shift_n += 5
    shift_h += 5
    shift_d += 5
    #continue
    
    merged_ = []
    for item in hal_chun:
        id_, src, tgt = item[0], item[1][0][0], item[1][0][1]
        merged_.append(['ced', id_, src, tgt])
        
    for item in hal_diff_chun:
        id_, src, tgt = item[0], item[1][0][0], item[1][0][1]
        merged_.append(['ced_gpt', id_, src, tgt])
        
    #for item in orig_neg_chun:
    #    id_, src, tgt = item[0], item[1][0][0], item[1][0][1]
    #    merged_.append(['original_neg', id_, src, tgt])
    
    for item in orig_chun:
        id_, src, tgt = item[0], item[1][0][0], item[1][0][1]
        merged_.append(['original', id_, src, tgt])    
        
    #print(merged_)
    
        
    for c, i in enumerate(merged_):
        c+=1
        id_, lbl, source_text, target_text = i[1], i[0], i[3], i[2]
     
        meta_id_file = open(f'{meta_id}_{c}.txt', 'w')
        meta_lbl_file = open(f'{meta_lbl}_{c}.txt', 'w')
        out_src_file = open(f'{out_src}_{c}.txt', 'w')
        out_tgt_file = open(f'{out_tgt}_{c}.txt', 'w')

        meta_id_file.write(f'{id_}')
        meta_lbl_file.write(f'{lbl}')
        out_src_file.write(f'{source_text}')
        out_tgt_file.write(f'{target_text}')

        meta_id_file.close()
        meta_lbl_file.close()
        out_src_file.close()
        out_tgt_file.close()
    


------

------


In [43]:
merged_

[['ced',
  337,
  ' <h-l-1>We have a mission and we have this issue already resolved to ensure</h-l-1> that at the least 30 % of those all <h-l-2>personnel</h-l-2> hired by the national government are women .',
  ' <h-l-1>Já decidimos garantir</h-l-1> que pelo menos 30 % de todos os <h-l-2>trabalhadores</h-l-2> contratados pelo governo nacional são mulheres .'],
 ['ced',
  351,
  ' Western correspondents are aware of these risks <h-l-2>when</h-l-2> they travel abroad to <h-l-1>avoid</h-l-1> active war zones .',
  ' Os correspondentes ocidentais estão conscientes destes riscos <h-l-2>, sempre</h-l-2> que viajam para o estrangeiro para <h-l-1>fazer a cobertura de</h-l-1> zonas de guerra em actividade .'],
 ['ced',
  356,
  ' In fact , I predict that 2018 will be a <h-l-1>breakout</h-l-1> year for many - though not all - Southeast Asian economies , owing to gains in eight key areas .',
  ' Com efeito , prevejo que 2018 seja um ano <h-l-1>determinante</h-l-1> para muitas - embora não todas

In [7]:
hal_chun

[]

In [5]:
7 + 7 + 11

42

In [19]:
10/25

0.4

In [20]:
6*25 

150